In [1]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
import os

In [2]:
graph = Neo4jGraph(
    url="YOUR_URL", username="USER_NAME", password="PASSWORD"
)

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "OPEN_API_KEY"

In [11]:
from langchain.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """
Task: Generate Cypher statement to query a graph database based on the user's question.
Instructions:
- Only use relationship types and properties provided in the schema.
- Construct Cypher statements to retrieve all requested information in a single query.
- Respond in Korean if the data is not available in the database.
- Do not include any explanations or apologies in your responses.
- The output should only include the Cypher statement.
- If the node has a 'detail' attribute, the response must include this 'detail' content.
Schema:
{schema}
Examples:
# Example of how many people played in "Top Gun":
MATCH (m:Movie {{title:"Top Gun"}})<-[:ACTED_IN]-() 
RETURN count(*) AS numberOfActors

# Example of the address of "홍성캠퍼스":
MATCH (c:Campus {{name:"홍성캠퍼스"}})-[:LOCATED_AT]->(a:Address) 
RETURN a.name as address

# Example of describing the "컴퓨터공학과":
MATCH (d:Department {{name:"컴퓨터공학과"}})-[:HAS_DESCRIPTION]->(desc:Description) 
RETURN desc.name AS Description

# 인천캠퍼스의 주소와 홍성캠퍼스의 주소가 뭐야?
MATCH (c1:Campus {{name:"인천캠퍼스"}})-[:LOCATED_AT]->(a1:Address),
      (c2:Campus {{name:"홍성캠퍼스"}})-[:LOCATED_AT]->(a2:Address)
RETURN a1.name AS Incheon_Campus_Address, a2.name AS Hongseong_Campus_Address

# 컴퓨터공학과는 어디에 있니?
MATCH (d:Department {{name:"컴퓨터공학과"}})-[:LOCATED_AT]->(l:Location)
RETURN l.name AS location

# 홍성캠퍼스의 도서관의 전화번호를 알려줘!
MATCH (c:Campus {{name:"홍성캠퍼스"}})-[:INCLUDES_LIBRARY]->(l:Library)-[:HAS_PHONE_NUMBER]->(pn:PhoneNumber)
RETURN pn.number AS Library_PhoneNumber

# 입학처의 전화번호가 뭐야?
MATCH (admission:Department {{name:"입학처"}})-[:HAS_PHONE_NUMBER]->(phone:PhoneNumber)
RETURN phone.number AS PhoneNumber

# 사무처의 전화번호를 알려줘!
MATCH (office:Department {{name:"사무처"}})-[:HAS_PHONE_NUMBER]->(phone:PhoneNumber)
RETURN phone.number AS PhoneNumber

# 청운학사(기숙사)의 전화번호가 뭐야?
MATCH (dormitory:Department {{name:"청운학사(기숙사)"}})-[:HAS_PHONE_NUMBER]->(phone:PhoneNumber)
RETURN phone.number AS PhoneNumber

# 인천캠퍼스의 교학처 전화번호가 뭐야?
MATCH (c:Campus {{name:"인천캠퍼스"}})-[:INCLUDES_TEACHINGCENTER]->(tc:EducationCenter {{name:"교학처"}})-[:HAS_PHONE_NUMBER]->(tp:PhoneNumber)
RETURN tp.number AS TeachingCenterPhoneNumber

# 인천캠퍼스의 사무처 전화번호가 뭐야?
MATCH (c:Campus {{name:"인천캠퍼스"}})-[:INCLUDES_ADMINISTRATIVECENTER]->(ac:EducationCenter {{name:"교학처"}})-[:HAS_PHONE_NUMBER]->(tp:PhoneNumber)
RETURN ac.number AS AdministrativeCenterPhoneNumber

# elcass 웹페이지 주소가 뭐야?
MATCH (:Cwuwebpage)-[:OFFERS_EClass]->(w:Webpage)
RETURN w.detail AS EClass_Webpage_Address

# 수강철회 신청서는 어떻게 작성해?
MATCH (c:CourseWithdrawal)-[:HAS_WITHDRAWAL_PROCEDURE_DOCUMENT]->(d:WithdrawalDetail)
RETURN d.detail as document_content

# 2024학번 학생이 졸업하기 위한 총 학점이 얼마야?
MATCH (g:StudentGrade {{name: 2024}})-[:REQUIRES_TOTAL_CREDITS]->(tc:TotalCredits)
RETURN tc.name AS TotalCreditsRequired



The question is:
{question}
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
)

In [5]:
chain.run("홍성캠퍼스의 주소가 뭐야?")

C:\Users\bjm\anaconda3\envs\torch-project\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Campus {name:"홍성캠퍼스"})-[:LOCATED_AT]->(a:Address) 
RETURN a.name as address
Full Context:
[{'address': '충남 홍성군 홍성읍 대학1길 66'}]

> Finished chain.


'충남 홍성군 홍성읍 대학1길 66'

In [15]:
chain.run("수강철회 신청서는 어떻게 작성해?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CourseWithdrawal)-[:HAS_WITHDRAWAL_PROCEDURE_DOCUMENT]->(d:WithdrawalDetail)
RETURN d.detail as document_content
Full Context:
[{'document_content': '홈페이지 http://home.chungwoon.ac.kr/ → 청운알림 → 서식자료실\r\n→ 학사서식 → 49번(수강과목 철회 신청서)'}]

> Finished chain.


'수강철회 신청서는 홈페이지 http://home.chungwoon.ac.kr/의 청운알림 → 서식자료실 → 학사서식에서 49번을 확인하시면 작성 방법을 찾을 수 있습니다.'

In [12]:
chain.run("이클래스 웹페이지 주소가 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Cwuwebpage)-[:OFFERS_EClass]->(w:Webpage)
RETURN w.detail AS EClass_Webpage_Address
Full Context:
[{'EClass_Webpage_Address': 'https://cyber.chungwoon.ac.kr/ilos/main/main_form.acl'}]

> Finished chain.


'https://cyber.chungwoon.ac.kr/ilos/main/main_form.acl'

In [11]:
chain.run("eclass 웹페이지 주소가 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Cwuwebpage)-[:OFFERS_EClass]->(w:Webpage)
RETURN w.detail AS EClass_Webpage_Address
Full Context:
[{'EClass_Webpage_Address': 'https://cyber.chungwoon.ac.kr/ilos/main/main_form.acl'}]

> Finished chain.


'eclass 웹페이지 주소는 https://cyber.chungwoon.ac.kr/ilos/main/main_form.acl 입니다.'

In [16]:
chain.run("컴퓨터공학과에 대해 설명해줘.")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Department {name:"컴퓨터공학과"})-[:HAS_DESCRIPTION]->(desc:Description) 
RETURN desc.name AS Description
Full Context:
[{'Description': '컴퓨터공학과에서는 컴퓨터 시스템 및 소프트웨어 개발 능력과 현장 실무능력을 지닌 전문 엔지니어를 양성하고 있습니다.학과 특성을 지향하는 학술 동아리에 모든 학년이 함께 활동하고 있고, IT관련 전문 지식을 습득하고 관련 분야의 국제공인자격증 등을 취득하고 있습니다.Cisco와 Java 학술동아리는 매년 CCNA(네트워크 국제공인자격증)와 SCJP(자바프로그래머 국제공인자격증)를 취득하고 있으며, CNAP(Cisco Networking Academy Program) 프로그램을 운영하고 있습니다.소프트웨어 개발, 서버시스템 모바일 등의 프로그램을 산학협력과 실무중심으로 교육하여 매우 높은 취업률달성에 노력하고 있습니다. 실무경력이 풍부한 교수진들이 최신IT시설을 갖춘 다양한 교육시설과 최고 수준의 연구 환경을 바탕으로 현장 실습형 교육을 제공합니다.'}]

> Finished chain.


'컴퓨터공학과에서는 컴퓨터 시스템 및 소프트웨어 개발 능력과 현장 실무능력을 지닌 전문 엔지니어를 양성하고 있습니다. 학과 특성을 지향하는 학술 동아리에 모든 학년이 함께 활동하고 있고, IT관련 전문 지식을 습득하고 관련 분야의 국제공인자격증 등을 취득하고 있습니다. Cisco와 Java 학술동아리는 매년 CCNA(네트워크 국제공인자격증)와 SCJP(자바프로그래머 국제공인자격증)를 취득하고 있으며, CNAP(Cisco Networking Academy Program) 프로그램을 운영하고 있습니다. 소프트웨어 개발, 서버시스템 모바일 등의 프로그램을 산학협력과 실무중심으로 교육하여 매우 높은 취업률달성에 노력하고 있습니다. 실무경력이 풍부한 교수진들이 최신IT시설을 갖춘 다양한 교육시설과 최고 수준의 연구 환경을 바탕으로 현장 실습형 교육을 제공합니다.'

In [17]:
chain.run("학부생의 정기주차 요금은 얼마야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:SubscriptionParking)-[:UNDERGRADUATE_FEE]->(sf:SubscriptionFee)
RETURN sf.name AS Undergraduate_SubscriptionFee
Full Context:
[{'Undergraduate_SubscriptionFee': '5000원/월 혹은 20,000원/학기'}]

> Finished chain.


'학부생의 정기주차 요금은 5000원/월 혹은 20,000원/학기 입니다.'

In [18]:
chain.run("인천캠퍼스의 주소와 홍성캠퍼스의 주소가 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c1:Campus {name:"인천캠퍼스"})-[:LOCATED_AT]->(a1:Address),
      (c2:Campus {name:"홍성캠퍼스"})-[:LOCATED_AT]->(a2:Address)
RETURN a1.name AS Incheon_Campus_Address, a2.name AS Hongseong_Campus_Address
Full Context:
[{'Incheon_Campus_Address': '인천광역시 미추홀구 숙골로 113(도화동)', 'Hongseong_Campus_Address': '충남 홍성군 홍성읍 대학1길 66'}]

> Finished chain.


'인천캠퍼스의 주소는 인천광역시 미추홀구 숙골로 113(도화동)이고, 홍성캠퍼스의 주소는 충남 홍성군 홍성읍 대학1길 66입니다.'

In [19]:
chain.run("강의평가는 어떻게 활용되니?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:CourseEvaluation)-[:UTILIZES_EVALUATION]->(ce:CourseEvaluationDetail)
RETURN ce.detail as Evaluation_Utilization_Method
Full Context:
[{'Evaluation_Utilization_Method': '수업 진행에 대한 학생들의 피드백을 확인함으로써 수업 개선 정보로 활용, 교수님 업적평가 및 다음 학기 수업개설에 중요한 자료로 활용, 학과 및 우리 대학 수업의 질을 높이는 데 활용'}]

> Finished chain.


'수업 진행에 대한 학생들의 피드백을 확인함으로써 수업 개선 정보로 활용, 교수님 업적평가 및 다음 학기 수업개설에 중요한 자료로 활용, 학과 및 우리 대학 수업의 질을 높이는 데 활용됩니다.'

In [16]:
chain.run("식당은 어디에 위치해 있니?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (r:Restaurant)-[:LOCATED_AT]->(a:Address)
RETURN r.name, a.name as detail
Full Context:
[]

> Finished chain.


"I don't know the answer."

In [21]:
chain.run("학생증은 어디에서 사용하나요?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:StudentID)-[:WHERE_TO_USE]->(d:Description)
RETURN d.name as Usage_Location
Full Context:
[{'Usage_Location': '1.학생신분 확인, 2.도서관 대출실 이용, 3.통학버스 이용, 4.기숙사 이용'}]

> Finished chain.


'학생증은 1.학생신분 확인, 2.도서관 대출실 이용, 3.통학버스 이용, 4.기숙사 이용에서 사용됩니다.'

In [5]:
chain.run("컴퓨터공학과는 어디에 있니?")

C:\Users\bjm\anaconda3\envs\torch-project\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Department {name:"컴퓨터공학과"})-[:LOCATED_AT]->(l:Location)
RETURN l.name AS location
Full Context:
[{'location': '인천 834호'}]

> Finished chain.


'인천 834호에 있습니다.'

In [12]:
chain.run("2018학번의 졸업학점의 전공학점을 알려줘!")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:StudentID {name:"2018학번"})-[:REQUIRES_TOTAL_CREDITS]->(tc:TotalCredits),
      (s)-[:REQUIRES_MAJOR_CREDITS]->(mc:MajorCredits)
RETURN mc.name AS MajorCredits
Full Context:
[]

> Finished chain.


'알 수 없습니다.'

In [5]:
chain.run("홍성캠퍼스 도서관의 전화번호가 뭐야?")

C:\Users\bjm\anaconda3\envs\torch-project\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Campus {name:"홍성캠퍼스"})-[:INCLUDES_LIBRARY]->(l:Library)-[:HAS_PHONE_NUMBER]->(pn:PhoneNumber)
RETURN pn.number AS Library_PhoneNumber
Full Context:
[{'Library_PhoneNumber': '041-630-3124, 041-630-3153, 041-630-3178, 041-630-3155'}]

> Finished chain.


'홍성캠퍼스 도서관의 전화번호는 041-630-3124, 041-630-3153, 041-630-3178, 041-630-3155 입니다.'

In [6]:
chain.run("인천캠퍼스 도서관의 전화번호를 알려줘!")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Campus {name:"인천캠퍼스"})-[:INCLUDES_LIBRARY]->(l:Library)-[:HAS_PHONE_NUMBER]->(pn:PhoneNumber)
RETURN pn.number AS Library_PhoneNumber
Full Context:
[{'Library_PhoneNumber': '032-770-8141, 032-770-8142, 032-770-8143'}]

> Finished chain.


'인천캠퍼스 도서관의 전화번호는 032-770-8141, 032-770-8142, 032-770-8143 입니다.'

In [9]:
chain.run("인천캠퍼스 사무처의 전화번호가 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Campus {name:"인천캠퍼스"})-[:INCLUDES_ADMINISTRATIVECENTER]->(ac:EducationCenter {name:"사무처"})-[:HAS_PHONE_NUMBER]->(phone:PhoneNumber)
RETURN phone.number AS PhoneNumber
Full Context:
[{'PhoneNumber': '032-770-8121'}]

> Finished chain.


'032-770-8121'

In [5]:
chain.run("홍성캠퍼스 기숙사 전화번호가 뭐야?")

C:\Users\bjm\anaconda3\envs\torch-project\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Campus {name:"홍성캠퍼스"})-[:INCLUDES_DORMITORY]->(d:Department {name:"청운학사(기숙사)"})-[:HAS_PHONE_NUMBER]->(pn:PhoneNumber)
RETURN pn.number AS Dormitory_PhoneNumber
Full Context:
[{'Dormitory_PhoneNumber': '041-630-3415'}]

> Finished chain.


'041-630-3415'

In [5]:
chain.run("수강철회장소는 어디에 있니?")

C:\Users\bjm\anaconda3\envs\torch-project\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CourseWithdrawal)-[:HAS_WITHDRAWAL_LOCATION]->(w:WithdrawalDetail)
RETURN w.name AS Withdrawal_Location
Full Context:
[{'Withdrawal_Location': '교무연구(교학)처(홍성-대학본부 129호, 인천-112호)'}]

> Finished chain.


'교무연구(교학)처(홍성-대학본부 129호, 인천-112호) 에 있습니다.'

In [9]:
chain.run("사회봉사 인증서 제출방법이 뭐야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:VolunteeringRequirement {name:"사회봉사"})-[:REQUIRES_CERTIFICATE_SUBMISSION]->(csd:CommunityServiceDetail)
RETURN csd.detail AS Submission_Method
Full Context:
[{'Submission_Method': '개별봉사활동입력 후 봉사활동확인서 업로드 - 통합정보시스템 로그인 → C.E.P(인재양성) 시스템 → 역량개발 → 교외활동 → 봉사활동 → 등록하기(봉사활동 일자별 내용 작성 후 확인서 및 인증서 첨부) - 사회복지자원봉사인증관리(VMS), 1365자원봉사자포털을 통해 봉사활동을 진행하고 봉사활동 확인서를 PDF로 내려받아 봉사활동 등록 시 첨부'}]

> Finished chain.


'개별봉사활동입력 후 봉사활동확인서 업로드 - 통합정보시스템 로그인 → C.E.P(인재양성) 시스템 → 역량개발 → 교외활동 → 봉사활동 → 등록하기(봉사활동 일자별 내용 작성 후 확인서 및 인증서 첨부) - 사회복지자원봉사인증관리(VMS), 1365자원봉사자포털을 통해 봉사활동을 진행하고 봉사활동 확인서를 PDF로 내려받아 봉사활동 등록 시 첨부.'

In [12]:
chain.run("2024 학생의 졸업학점이 총 몇학점을 들어야 하니?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (g:StudentGrade {name: 2024})-[:REQUIRES_TOTAL_CREDITS]->(tc:TotalCredits)
RETURN tc.name AS TotalCreditsRequired
Full Context:
[{'TotalCreditsRequired': 130}]

> Finished chain.


'2024 학생의 졸업학점이 총 130학점을 들어야 합니다.'

In [13]:
chain.run("18학번 학생이 졸업을 위해 총 몇 학점을 들어야 하니?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (g:StudentGrade {name: 2018})-[:REQUIRES_TOTAL_CREDITS]->(tc:TotalCredits)
RETURN tc.name AS TotalCreditsRequired
Full Context:
[{'TotalCreditsRequired': '130, 간호학과 학생은 140'}]

> Finished chain.


'18학번 학생이 졸업을 위해 총 130 학점을 들어야 합니다.'

In [14]:
chain.run("20학번 학생이 졸업을 위해 교양과목을 총 몇 학점을 들어야 하니?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (g:StudentGrade {name: 2020})-[:REQUIRES_ELECTIVE_CREDITS]->(ec:ElectiveCredits)
RETURN ec.name AS RequiredElectiveCredits
Full Context:
[{'RequiredElectiveCredits': '33~42'}]

> Finished chain.


'20학번 학생이 졸업을 위해 교양과목을 총 33~42 학점을 들어야 합니다.'